# Advanced SQL Querying

**Purpose**: Complex queries and joins
**Data**: 2022-2023

Learn raw SQL for advanced filtering.

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent / 'src'))

from maude_db import MaudeDatabase
import pandas as pd

In [ ]:
db = MaudeDatabase('advanced.db', verbose=True)
db.add_years(years='2022-2023', tables=['device', 'master', 'patient'], download=True, data_dir='./maude_data')

## Complex Filters

In [ ]:
# Multiple device types with date filter - using patient outcomes for accurate death counts
sql = """
    SELECT m.*, d.GENERIC_NAME, d.BRAND_NAME, p.SEQUENCE_NUMBER_OUTCOME
    FROM master m
    JOIN device d ON m.MDR_REPORT_KEY = d.MDR_REPORT_KEY
    LEFT JOIN patient p ON m.MDR_REPORT_KEY = p.MDR_REPORT_KEY
    WHERE (d.GENERIC_NAME LIKE '%pacemaker%' OR d.GENERIC_NAME LIKE '%defibrillator%')
      AND m.DATE_RECEIVED >= '2023-01-01'
      AND p.SEQUENCE_NUMBER_OUTCOME LIKE '%D%'
"""
results = db.query(sql)
print(f'Found {len(results):,} fatal events')

## Aggregations

In [4]:
# Top manufacturers by event count
sql = """
    SELECT d.MANUFACTURER_D_NAME, COUNT(*) as events
    FROM device d
    WHERE d.GENERIC_NAME LIKE '%catheter%'
    GROUP BY d.MANUFACTURER_D_NAME
    ORDER BY events DESC
    LIMIT 10
"""
top_mfg = db.query(sql)
print(top_mfg)

                              MANUFACTURER_D_NAME  events
0                   BOSTON SCIENTIFIC CORPORATION   13660
1            C.R. BARD, INC. (COVINGTON) -1018233    9422
2              C.R. BARD, INC. (BASD) -3006260740    9308
3                            BIOSENSE WEBSTER INC    5938
4                         ARROW INTERNATIONAL LLC    4018
5  BECTON DICKINSON INFUSION THERAPY SYSTEMS INC.    2274
6                                ST. JUDE MEDICAL    2203
7                                 ABBOTT VASCULAR    2157
8                                  ABBOTT MEDICAL    1710
9                MICRO THERAPEUTICS, INC. DBA EV3    1498


In [5]:
db.close()
print('See docs/api_reference.md for more!')

See docs/api_reference.md for more!
